In [ ]:
import numpy as np
import pandas as pd
import regex as re
import random as rn
import ast
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,average_precision_score, precision_score,precision_recall_curve
from tqdm.notebook import tqdm
from tqdm import trange
import warnings
import pickle
import nltk
import math
import os
import json
import random
import re
import torch
import torch.nn as nn
import spacy 
from spacy import displacy
#import neptune.new as neptune                          
from torch.utils.data import (DataLoader, RandomSampler, WeightedRandomSampler, SequentialSampler, TensorDataset, Dataset)

In [ ]:
!pip install emoji -q
!pip install datasets -q
!pip install rouge_score -q


!pip install transformers -q
!pip install sentencepiece -q
!pip install pandarallel -q

from transformers import AdamW, get_linear_schedule_with_warmup
from transformers import AutoTokenizer, AutoModel

from transformers import AdamW, get_linear_schedule_with_warmup,PegasusConfig, PegasusTokenizer, PegasusForConditionalGeneration
from transformers import AutoTokenizer, AutoModel, set_seed, TrainingArguments, Trainer, default_data_collator, AdamW, get_linear_schedule_with_warmup, Seq2SeqTrainer, Seq2SeqTrainingArguments


     |████████████████████████████████| 170 kB 5.1 MB/s 
     |████████████████████████████████| 298 kB 5.1 MB/s 
     |████████████████████████████████| 243 kB 58.3 MB/s 
     |████████████████████████████████| 132 kB 61.5 MB/s 
     |████████████████████████████████| 1.1 MB 48.1 MB/s 
     |████████████████████████████████| 61 kB 602 kB/s 
     |████████████████████████████████| 192 kB 61.2 MB/s 
     |████████████████████████████████| 160 kB 51.0 MB/s 
     |████████████████████████████████| 271 kB 66.3 MB/s 
     |████████████████████████████████| 3.4 MB 5.1 MB/s 
     |████████████████████████████████| 596 kB 70.2 MB/s 
     |████████████████████████████████| 3.3 MB 15.8 MB/s 
     |████████████████████████████████| 895 kB 69.2 MB/s 
     |████████████████████████████████| 1.2 MB 5.1 MB/s 


In [ ]:
from transformers import PegasusForConditionalGeneration, PegasusTokenizer
import torch
model_name = 'google/pegasus-xsum'
device = 'cuda' if torch.cuda.is_available() else 'cpu'
config = PegasusConfig.from_pretrained(model_name)
tokenizer = PegasusTokenizer.from_pretrained(model_name)
model = PegasusForConditionalGeneration.from_pretrained(model_name).to(device)

Downloading:   0%|          | 0.00/1.36k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.82M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/87.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.36M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.12G [00:00<?, ?B/s]

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
device = 'cuda' if torch.cuda.is_available() else 'cpu'

Mounted at /content/drive


In [ ]:
reviews_path = '/content/drive/MyDrive/gen_datasets/hotel_travel_reviews.json'
# reviews_path = '/content/drive/MyDrive/restaurant_reviews/res_reviews_final.json'
import json
f = open(reviews_path)
# data = json.load(f)
data = json.load(f)
x = data

In [ ]:
# x = {}
# for key_ in data.keys():
#   for a in data[key_]['review_list']:
#     if(a['rating']!=None):
#       x[a['text']] = a['rating']

In [ ]:
sorted_vals = sorted(x.items(), key=lambda item: item[1])

In [ ]:
reviews = []
for j in sorted_vals:
  reviews.append(j[0])

In [ ]:
num_kp = int(len(reviews)/50)
input = []
for i in range(len(reviews)):
  if(i%num_kp==0):
    input.append(reviews[i])
    src_text = ''.join(input)
    batch = tokenizer(src_text, truncation=True, padding='longest', return_tensors="pt").to(device)
    translated = model.generate(**batch)
    tgt_text = tokenizer.batch_decode(translated, skip_special_tokens=True)
    print(i,tgt_text)
    input = []

0 ['The food at Pop Century sucks!']
71 ["I stayed at this hotel for a week and it's not good at all."]
142 ['I booked a car from Avis on the day of my flight.']
213 ['Budget Rent-a-Truck.com:']
284 ["Budget's car rental counter at Orlando International Airport is so long it's almost comical."]
355 ['I went to Budget today to rent a car to take to work.']
426 ['Customer service is terrible.']
497 ['I booked through a third party and should have had advantage when I saw the red flags. I booked through a third party and should have had advantage when I saw the red flags.']
568 ['This was my first time staying at this motel.']
639 ["This is the worst business I've ever experienced."]
710 ['When my husband and I decided to rent a car from Hertz, we had no idea what to expect.']
781 ['I have been writing to you to express my disappointment in your sales staff.']
852 ['I have never been to a brewery before.']
923 ['Has anyone been affected by a fungus?']
994 ["I'm staying at a hotel in Londo